<a href="https://colab.research.google.com/github/zyang63/Gate_design/blob/main/gating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geometric Analysis in Google Colab


The code installs and imports several libraries for different purposes. The trimesh library is installed to work with triangular meshes, SimpleITK is installed for medical image analysis, scipy is installed for scientific computing, and bpy is installed for 3D modeling and rendering. The directory "geometry" is created to store the various stl files generated by the analysis. The result of the code is stored in the base directory and produces three "gltf" animations which can be downloaded and reviewed.

In [1]:
%%capture
try:
  import open3d as o3d
except ImportError:
  !pip install open3d
  import open3d as o3d
try:
  import SimpleITK as sitk
except ImportError:
  !pip install SimpleITK
  import SimpleITK as sitk
try:
  import bpy
except ImportError:
  !pip install bpy
  import bpy
try:
  from stl import mesh
except ImportError:
  !pip install numpy-stl
  from stl import mesh
try:
  import pyvista as pv
except ImportError:
  !pip install pyvista
  import pyvista as pv
import numpy as np
from scipy import ndimage
from skimage import measure
import os

#Setup

In [2]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
#@title #File Entry { display-mode: "form"}
#@markdown User can choose to upload the file to colab directly or select the google file upload button at the bottom of this form. Also choose the number of elements.

button_pressed = False  # Initialize the variable as False
filename = ""
button = widgets.Button(description="Google upload dialog")
output = widgets.Output()

def on_button_clicked(b):
    global button_pressed  # Access the global variable
    global filename
    with output:
        uploaded = files.upload()
        filename = list(uploaded.keys())[0]
        button_pressed = True  # Set the variable to True when the button is clicked

button.on_click(on_button_clicked)
display(button, output)

Button(description='Google upload dialog', style=ButtonStyle())

Output()

In [3]:
#@title #Element Count { display-mode: "form", run: "auto" }
element_count = 200 #@param {type:"slider", min:10, max:500, step:1}
if not button_pressed:
  filename = "/content/steering_column_mount.stl" #@param {type:"string"}
geometry = o3d.io.read_triangle_mesh(filename)
max_boundary_size = (geometry.get_max_bound()-geometry.get_min_bound()).max()
print(max_boundary_size)
voxel_resolution = max_boundary_size/element_count
print("File used ", filename)
print("Element size is in units from stl file ",voxel_resolution, " per cell")

151.66339111328125
File used  /content/steering_column_mount.stl
Element size is in units from stl file  0.7583169555664062  per cell


# Voxelizing the geometry


The following code performs voxelization and mesh conversion operations. It starts by voxelizing a mesh with a specified voxel size. Then, it creates a solid voxel representation by filling the voxel surface. The solid voxel data is padded with two layers of zeros. The padded voxel data is further processed using marching cubes algorithm to obtain a mesh representation. Finally, the resulting mesh is exported as an STL file named "marching_geometry.stl" in the geometry directory.

In [4]:
pcd = geometry.sample_points_uniformly(number_of_points=100000000)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=voxel_resolution)
voxels = voxel_grid.get_voxels()
indices = np.stack(list(vx.grid_index for vx in voxels))
del voxels, voxel_grid, pcd
max_indices = np.max(indices, axis=0)+1
dense_array = np.zeros(max_indices, dtype=np.bool8)
for idx in indices:
    dense_array[tuple(idx)] = 1
del indices
array_pad = np.pad(dense_array.astype(bool),((2,2)),'constant')
del dense_array
array_closing = ndimage.binary_closing(array_pad, structure=ndimage.generate_binary_structure(3, 1), iterations=1, mask=None,  border_value=0, origin=0, brute_force=False)
img = sitk.GetImageFromArray(array_closing.astype(int))
seg = sitk.ConnectedComponent(img != img[0,0,0])
img_filled = sitk.BinaryFillhole(seg!=0)
array_filled = sitk.GetArrayFromImage(img_filled)
verts, faces, _, _ = measure.marching_cubes(array_filled)
vertices_original = np.asarray(verts.tolist())
faces_original = np.asarray(faces.tolist())

# Morphological operations and Distance transform on mesh


The following code applies binary closing to a padded voxel data array and assigns the result to array_initial for further use. It then defines a function named core that performs operations to extract the core of a structure from an input array. The function is applied to array_initial, and the resulting array is assigned to array_core. The code further manipulates array_core by setting non-core elements to 0 and core elements to 1. Overall, these operations help identify and isolate the core of a structure within the voxel data.

In [5]:
array_initial = ndimage.binary_closing(array_filled, structure=ndimage.generate_binary_structure(3, 1), iterations=1, mask=None,  border_value=0, origin=0, brute_force=False)
img_initial = sitk.GetImageFromArray(array_initial.astype(int))

## Distance Field Calculation
The following code calculates the signed Maurer distance map from a binary voxel data representation. The distance map is computed using the SignedMaurerDistanceMap function from the SimpleITK library. The resulting distance map is then converted into a NumPy array. The code further determines the depth of the object by taking the negative minimum value from the distance map and converting it to an integer. This depth value represents the distance inside the object. Overall, these operations provide information about the spatial distribution and depth of the object in the voxel data.

In [6]:
img_dist = sitk.SignedMaurerDistanceMap(img_initial != 0, insideIsPositive=False, squaredDistance=False, useImageSpacing=False)
array_dist = sitk.GetArrayFromImage(img_dist)
depth = int(-array_dist.min())
print(depth)

9


## Parting Line Analysis

In [7]:
def sum_3d_array_along_height(input_array):
    height, rows, cols = input_array.shape
    result_array = np.sum(input_array, axis=2)

    return result_array
def find_edge(array):
    rows, cols = array.shape
    output = np.zeros((rows, cols), dtype=int)

    for i in range(rows):
        for j in range(cols):
            if array[i, j] != 0:
                if (i == 0 or array[i - 1, j] == 0) or \
                   (i == rows - 1 or array[i + 1, j] == 0) or \
                   (j == 0 or array[i, j - 1] == 0) or \
                   (j == cols - 1 or array[i, j + 1] == 0):
                    output[i, j] = 1

    return output

In [8]:
def find_overlap_3d_2d(arr_3d, arr_2d):
    overlap_arr = np.logical_and(arr_3d, arr_2d[:, :, np.newaxis]).astype(int)
    return overlap_arr

In [9]:
def extract_max_ones_layers(arr_3d):
    rows, cols, height = arr_3d.shape
    # Sum along the (rows, cols) axes to count the number of ones in each layer
    ones_count = np.sum(arr_3d, axis=(0, 1))

    # Find the maximum ones count
    max_ones_count = np.max(ones_count)

    # Find the indices of the layers with the maximum ones count
    layer_indices = np.where(ones_count == max_ones_count)[0]

    # Create a new 3D array with the same shape as the input array
    result_arr = np.zeros_like(arr_3d)

    # Set the layers with the maximum ones to the result array
    result_arr[:, :, layer_indices] = arr_3d[:, :, layer_indices]

    # Find the median layer index
    #median_layer_index = np.median(layer_indices).astype(int)

    # Create a new 3D array containing only the median layer
    #median_layer_arr = np.zeros_like(arr_3d)
    #median_layer_arr[:, :, median_layer_index] = arr_3d[:, :, median_layer_index]

    return result_arr

In [10]:
# Step 1
S = sum_3d_array_along_height(array_filled)
B = find_edge(S)
N = find_overlap_3d_2d(array_filled, B)
C = extract_max_ones_layers(N)
# Step 2
T = find_edge(S)
P = np.zeros_like(T)
M = np.zeros_like(C)
sum_P = np.count_nonzero(P == 1)
U = T - P
sum_U = np.count_nonzero(U == 1)
#print(sum_U)
last_sum_U = None

while sum_U >= 0:
    V = find_overlap_3d_2d(N, U)
    L = extract_max_ones_layers(V)
    M = np.where(L == 1, 1, M)
    H = sum_3d_array_along_height(L)
    I = find_edge(H)
    P = np.where(I == 1, 1, P)
    sum_P = np.count_nonzero(P == 1)
    #print(sum_P)

    U = np.where(P == 1, 0, T)
    sum_U = np.count_nonzero(U == 1)
    #print(sum_U)

    if last_sum_U == sum_U:
        break

    last_sum_U = sum_U

In [11]:
# Step 1: Count values for each layer
layer_counts = np.sum(M, axis=(0, 1))

# Step 2: Replace values based on count number
unique_counts, count_occurrences = np.unique(layer_counts, return_counts=True)
sorted_indices = np.argsort(count_occurrences)[::-1]

for i, count in enumerate(unique_counts[sorted_indices]):
    indices_to_replace = np.where(layer_counts == count)[0]

    for index in indices_to_replace:
        M[:, :, index][M[:, :, index] == 1] = i + 1

In [12]:
print(M.max())

9


# Gate located


In [13]:
img_thresh = img_filled != 0
img_ws = sitk.MorphologicalWatershed( img_dist, markWatershedLine=True, level=1)
img_ws = sitk.Cast(img_ws, img_thresh.GetPixelID())
img_ws_mark = sitk.Mask(img_ws, img_thresh)
array_ws_mark = sitk.GetArrayFromImage(img_ws_mark)

In [14]:
A = array_ws_mark.max()
print(A)

17


In [15]:
counts = np.zeros(A, dtype=int)

for i in range(1, A + 1):
    counts[i - 1] = np.count_nonzero(array_ws_mark == i)

print("Counts for values :", counts)


Counts for values : [  5237   5251   6027   9004  20712  21518 104566  15867  99472  37783
  38099  16763  16502  17755  13856   5145  13286]


In [16]:
unit = 0.01
injection_speed = 30.5 # m/s
injection_time = 0.5 #s
gate_size = np.array(((counts * ((voxel_resolution * unit) **3))/(injection_speed * injection_time)))
print(gate_size)

[0.00014975 0.00015015 0.00017234 0.00025746 0.00059225 0.0006153
 0.00299001 0.00045371 0.00284435 0.00108039 0.00108942 0.00047933
 0.00047187 0.0005077  0.00039621 0.00014712 0.00037991]


In [17]:
# Find coordinates of the minimum value in distance field
def find_coordinates(array_dist):
    indices = np.nonzero(array_dist == array_dist.min())
    if indices[0].size == 0:
          return None
    return indices[0][0], indices[1][0], indices[2][0]

In [18]:
origin = geometry.get_min_bound()

In [19]:
def find_closest_nonzero_element(array_1, array_2, target_value):
    # Find the coordinates of the target_value in array_1
    target_coords = np.argwhere(array_1 == target_value)
    if not target_coords.size:
        return None  # Target value not found in array_1

    target_coords = target_coords[0]  # Use the first occurrence if there are multiple

    # Create a boolean mask for non-zero elements in array_2
    nonzero_mask = (array_2 != 0)

    # Calculate the Euclidean distance between target_coords and all non-zero elements in array_2
    distances = np.linalg.norm(np.argwhere(nonzero_mask) - target_coords, axis=1)

    # Find the index of the minimum distance
    min_distance_index = np.argmin(distances)

    # Get the coordinates of the closest non-zero element
    closest_coords = tuple(np.argwhere(nonzero_mask)[min_distance_index])

    return closest_coords

In [20]:
E_list = []
X_list = []
coordinates = []
G = array_ws_mark.max() + 1
# Loop through all pairs of bounding boxes and check if they intersect
for i in range(1, G):
  F_i = np.where(array_ws_mark != i, 0, array_ws_mark)
  dilated_array_i = ndimage.binary_dilation(F_i, structure=ndimage.generate_binary_structure(3, 1)).astype(F_i.dtype)
  T_i = np.where(F_i != i, 0, array_dist)
  M_i = T_i.min()
  gate_coord = find_closest_nonzero_element(T_i, M, M_i)
  coordinates_i = find_coordinates(T_i)
  coordinates.append(gate_coord)
  H_i = ((coordinates_i[0] - 2)  * voxel_resolution-1 + origin[0],
           (coordinates_i[1] - 2) * voxel_resolution + origin[1],
           (coordinates_i[2] - 2) * voxel_resolution + origin[2])
  X_list.append(H_i)
  E_i = ((gate_coord[0] - 2) * voxel_resolution + origin[0],
           (gate_coord[1] - 2) * voxel_resolution + origin[1],
           (gate_coord[2] - 2) * voxel_resolution + origin[2])
  E_list.append(E_i)
print(X_list)
print(E_list)
print(coordinates)

[(2.033267822265625, 116.78081115722655, 56.11545471191406), (2.033267822265625, 125.88061462402344, 56.11545471191406), (36.157530822753905, 6.06653564453125, 97.0645703125), (36.157530822753905, 21.232874755859374, 97.0645703125), (37.67416473388672, 23.50782562255859, 61.423673400878904), (56.63208862304687, 115.26417724609374, 83.41486511230468), (58.90703948974609, 7.583169555664062, 26.541093444824217), (77.10664642333984, 27.299410400390624, 95.54793640136718), (61.18199035644531, 49.290602111816405, 31.090995178222656), (61.94030731201172, 56.87377166748047, 87.96476684570312), (75.59001251220702, 53.84050384521484, 86.44813293457031), (75.59001251220702, 66.73189208984374, 28.816044311523434), (75.59001251220702, 66.73189208984374, 87.20644989013671), (92.27298553466797, 80.38159729003905, 101.61447204589844), (99.09783813476562, 3.033267822265625, 72.798427734375), (100.61447204589844, 5.3082186889648435, 97.0645703125), (100.61447204589844, 21.232874755859374, 97.0645703125)

In [21]:
def get_values_at_positions(array, coordinates_list):
    values = [array[x, y, z] for x, y, z in coordinates_list]
    return values

# Get values at specified positions
values_at_positions = get_values_at_positions(M, coordinates)
locations = []
# Display the results
for i, position in enumerate(coordinates):
    locations.append(values_at_positions[i])
print(locations)

[7, 7, 3, 3, 7, 4, 8, 3, 8, 3, 3, 8, 3, 3, 7, 3, 3]


In [22]:
def count_consecutive_ones(matrix):
    row_count = []
    col_count = []

    # Count consecutive ones in each row
    for row in matrix:
        count = 0
        max_count = 0
        for element in row:
            if element == thickness:
                count += 1
                max_count = max(max_count, count)
            else:
                count = 0
        row_count.append(max_count)

    # Count consecutive ones in each column
    for col in range(len(matrix[0])):
        count = 0
        max_count = 0
        for row in matrix:
            if row[col] == thickness:
                count += 1
                max_count = max(max_count, count)
            else:
                count = 0
        col_count.append(max_count)

    return row_count, col_count

In [23]:
# List to store the resulting arrays
result_arrays = []
max_size = []
# Extracting arrays based on the random indices
for index in coordinates:
    result_array = M[index[0], :, :]
    result_arrays.append(result_array)
# Printing the results
for i, result_array in enumerate(result_arrays, 1):
    thickness = locations[i - 1]

    row_count, col_count = count_consecutive_ones(result_array)
    max_size.append((max(row_count), max(col_count)))

print(max_size)

[(1, 21), (1, 21), (4, 28), (4, 28), (1, 1), (1, 10), (1, 1), (4, 81), (1, 1), (4, 1), (4, 81), (1, 1), (4, 81), (4, 81), (1, 1), (4, 29), (4, 1)]


In [24]:
for index in max_size:
    Q = gate_size / (index[0] * (voxel_resolution * unit) ** 2)

print(Q)

[ 0.65103375  0.65277415  0.74924201  1.11932555  2.57479685  2.67499414
 12.99904439  1.97249428 12.36578757  4.6969655   4.7362488   2.08387986
  2.05143384  2.2071996   1.72249832  0.63959684  1.65163919]


In [25]:
W = (gate_size / ( (voxel_resolution * unit) ** 2)) ** 0.5
print(W)

[1.61373325 1.6158888  1.73117534 2.11596366 3.20923471 3.27108186
 7.21083751 2.80891031 7.03300436 4.33449674 4.35258489 2.88712996
 2.86456547 2.9713294  2.62487967 1.59949597 2.57032231]


In [30]:
new_value =  max_boundary_size * 0.01

In [31]:
for i in range(len(max_size)):
    if Q[i] > W[i]:
        max_index = max_size[i].index(max(max_size[i]))
        max_size[i] = tuple(val if j != max_index else Q[i] for j, val in enumerate(max_size[i]))

        # Display the result


        cuboid_sizes = [(item[0] * voxel_resolution, item[1]* voxel_resolution, new_value* voxel_resolution) for item in max_size]

    else:
        #replace_value = max_size[i]  # The value you want to replace in max_size[i]
        max_size[i] = tuple(W[i] for val in max_size[i])

        # Display the result

        cuboid_sizes = [(item[0] * voxel_resolution, item[1] * voxel_resolution, new_value * voxel_resolution) for item in max_size]
print(cuboid_sizes)

[(1.2237212880117119, 1.2237212880117119, 1.150089210199006), (1.2253558749470204, 1.2253558749470204, 1.150089210199006), (1.312779613813728, 1.312779613813728, 1.150089210199006), (1.6045711203876132, 1.6045711203876132, 1.150089210199006), (2.433617092063855, 2.433617092063855, 1.150089210199006), (2.4805168400889346, 2.4805168400889346, 1.150089210199006), (9.857395766694202, 0.7583169555664062, 1.150089210199006), (2.130044315738069, 2.130044315738069, 1.150089210199006), (9.37718638663242, 0.7583169555664062, 1.150089210199006), (3.561788576143364, 0.7583169555664062, 1.150089210199006), (3.033267822265625, 3.591577772079666, 1.150089210199006), (2.1893596043575543, 2.1893596043575543, 1.150089210199006), (2.1722485679308736, 2.1722485679308736, 1.150089210199006), (2.253209463006663, 2.253209463006663, 1.150089210199006), (1.9904907583122062, 1.9904907583122062, 1.150089210199006), (1.2129249168000653, 1.2129249168000653, 1.150089210199006), (1.9491189876181705, 1.94911898761817

In [32]:
# Step 1
for i in range(len(max_size)):
    max_index = max_size[i].index(max(max_size[i]))
    max_size[i] = tuple(val if j != max_index else Q[i] for j, val in enumerate(max_size[i]))

# Display the result
print(max_size)
new_value =  max_boundary_size * 0.01

cuboid_sizes = [(item[0] * voxel_resolution, item[1]* voxel_resolution, new_value* voxel_resolution) for item in max_size]

print(cuboid_sizes)

[(0.6510337534920114, 1.6137332536599862), (0.6527741530621638, 1.61588879946878), (0.7492420149506116, 1.7311753405713837), (1.11932555211802, 2.115963659534842), (2.5747968497854767, 3.209234706147543), (2.6749941393242507, 3.271081863435552), (12.999044389468335, 1), (1.9724942842577324, 2.8089103113184177), (12.36578757444288, 1), (4.696965497076317, 1), (4, 4.736248801659756), (2.0838798567474863, 2.88712996364728), (2.0514338361896454, 2.8645654722415723), (2.2071995977182857, 2.9713293979081388), (1.7224983174308401, 2.6248796676654265), (0.63959684203101, 1.5994959731503047), (1.651639192074635, 2.5703223082521265)]
[(0.49368993391903226, 1.2237212880117119, 1.150089210199006), (0.49500970842253933, 1.2253558749470204, 1.150089210199006), (0.5681629237597876, 1.312779613813728, 1.150089210199006), (0.8488035449698238, 1.6045711203876132, 1.150089210199006), (1.952512108331296, 2.433617092063855, 1.150089210199006), (2.028493411890345, 2.4805168400889346, 1.150089210199006), (9.

# Gate generated

In [33]:
# Function to calculate rotation matrix
def calculate_rotation_matrix(v, w):
    cos_theta = np.dot(v, w) / (np.linalg.norm(v) * np.linalg.norm(w))
    sin_theta = np.sqrt(1 - cos_theta**2)
    theta = np.arccos(cos_theta)

    u = np.cross(v, w) / np.linalg.norm(np.cross(v, w))

    # Rotation matrix formula
    R = np.array([
        [cos_theta + u[0]**2 * (1 - cos_theta), u[0]*u[1]*(1 - cos_theta) - u[2]*sin_theta, u[0]*u[2]*(1 - cos_theta) + u[1]*sin_theta],
        [u[1]*u[0]*(1 - cos_theta) + u[2]*sin_theta, cos_theta + u[1]**2 * (1 - cos_theta), u[1]*u[2]*(1 - cos_theta) - u[0]*sin_theta],
        [u[2]*u[0]*(1 - cos_theta) - u[1]*sin_theta, u[2]*u[1]*(1 - cos_theta) + u[0]*sin_theta, cos_theta + u[2]**2 * (1 - cos_theta)]
    ])

    return R

# Function to create cuboid vertices
def create_cuboid(cuboid_size):
    vertices = np.array([
        [-0.5, -0.5, 0] * cuboid_size,
        [0.5, -0.5, 0] * cuboid_size,
        [0.5, 0.5, 0] * cuboid_size,
        [-0.5, 0.5, 0] * cuboid_size,
        [-0.5, -0.5, 2] * cuboid_size,
        [0.5, -0.5, 2] * cuboid_size,
        [0.5, 0.5, 2] * cuboid_size,
        [-0.5, 0.5, 2] * cuboid_size
    ])
    return vertices

# Initialize an empty list to store all meshes
all_cuboid_meshes = []

# Iterate through each pair of points
for point_A, point_B, cuboid_size  in zip(X_list, E_list, cuboid_sizes):
    # Assuming point_A and point_B are lists with 3 coordinates
    point_A = np.array(point_A)
    point_B = np.array(point_B)
    # Define cuboid size
    cuboid_size = np.array(cuboid_size)
    # Original direction vector
    original_direction = np.array([0, 0, 1])

    # Target direction vector
    target_direction = (point_B - point_A) * np.array([1, 1, 0])

    # Calculate rotation matrix
    rotation_matrix = calculate_rotation_matrix(original_direction, target_direction)

    # Create cuboid
    cuboid_vertices = create_cuboid(cuboid_size)

    # Apply rotation matrix to cuboid vertices
    rotated_vertices = point_B + np.dot(cuboid_vertices, rotation_matrix.T)

    # Define faces for the cuboid
    faces = np.array([
        [0, 1, 2],
        [0, 2, 3],
        [4, 6, 5],
        [4, 7, 6],
        [0, 4, 5],
        [0, 5, 1],
        [2, 6, 7],
        [2, 7, 3],
        [0, 3, 7],
        [0, 7, 4],
        [1, 5, 6],
        [1, 6, 2]
    ])

    # Create STL mesh using faces
    cuboid_mesh = mesh.Mesh(np.zeros(len(faces), dtype=mesh.Mesh.dtype))
    for i, face in enumerate(faces):
        cuboid_mesh.vectors[i] = rotated_vertices[face]

    # Append the current mesh to the list
    all_cuboid_meshes.append(cuboid_mesh)

# Combine all meshes into a single mesh
combined_mesh = mesh.Mesh(np.concatenate([mesh.data for mesh in all_cuboid_meshes]))

# Save the single STL file
combined_mesh.save('all_gate.stl')


In [ ]:
bpy.ops.wm.read_factory_settings(use_empty=True)
bpy.ops.import_mesh.stl(filepath=filename)
obj = bpy.context.object
obj.select_set(True)
bpy.context.view_layer.objects.active = obj
obj.color = (0,0,1,1)
mat = bpy.data.materials.new("0")
mat.use_nodes = True
principled = mat.node_tree.nodes["Principled BSDF"]
principled.inputs["Base Color"].default_value = (1,1,1,1)
principled.inputs["Alpha"].default_value = 0.7
obj.data.materials.append(mat)
bpy.context.object.active_material.blend_method = "BLEND"
bpy.ops.import_mesh.stl(filepath="//content//all_gate.stl")
obj = bpy.context.object
obj.color = (0,0,1,1)
mat = bpy.data.materials.new("0")
mat.use_nodes = True
principled = mat.node_tree.nodes["Principled BSDF"]
principled.inputs["Base Color"].default_value = (0,1,0,1)
obj.data.materials.append(mat)
bpy.ops.export_scene.gltf(filepath="//content//colored_gating.glb")